In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

In [3]:
file = 'wine_catalogue.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [4]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [5]:
llm = ChatOpenAI(temperature = 0.0)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

In [10]:
data[0]


Document(page_content=': 0\ntitle: Fragolino Rosso Frizzante - Bottega\ntype: wine-based beverage\nkind: fragolino\ncountry: mild - doux - sweet - dolce\nregion: Italy\nprice_eur: 5.97\nvolume_lit: 0.75\norder_number: 70601-IT-RW-001', metadata={'source': 'wine_catalogue.csv', 'row': 0})

In [11]:
examples = [
    {
        "query": "Is Fleur de d'Artagnan Blanc Côtes de Gascogne 2020 - Plaimont\
        dry",
        "answer": "Yes"
    },
    {
        "query": "What kind is Fragolino Rosso Frizzante - Bottega?",
        "answer": "fragolino"
    }
]

In [12]:
from langchain.evaluation.qa import QAGenerateChain


In [13]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())

In [14]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

In [15]:
new_examples[:5]

[{'query': 'What is the name of the wine-based beverage mentioned in the document and what is its origin?',
  'answer': 'The name of the wine-based beverage mentioned in the document is Fragolino Rosso Frizzante and it originates from Italy.'},
 {'query': 'What is the title of the white wine mentioned in the document?',
  'answer': 'The title of the white wine mentioned in the document is "Grauer Burgunder eins zu eins Kabinett trocken 2021 - A. Diehl."'},
 {'query': 'What is the title of the red wine described in this document?',
  'answer': 'The title of the red wine described in this document is "Fabelhaft Tinto Douro DOC Rotwein 2020 - Niepoort".'},
 {'query': 'What is the title of the wine listed in the document?',
  'answer': 'The title of the wine listed in the document is "Motivo Rosé extra dry - Borgo Molino".'},
 {'query': 'What is the title and type of wine in the document?',
  'answer': 'The title of the wine is Mandus Primitivo di Manduria DOC 2019 - Pietra Pura and the ty

In [16]:
examples += new_examples

In [18]:
examples[0]

{'query': "Is Fleur de d'Artagnan Blanc Côtes de Gascogne 2020 - Plaimont        dry",
 'answer': 'Yes'}

In [17]:
qa.run(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


"Yes, Fleur de d'Artagnan Blanc Côtes de Gascogne 2020 - Plaimont is a dry white wine."

In [19]:
import langchain
langchain.debug = True

In [20]:
qa.run(examples[0]["query"])

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Is Fleur de d'Artagnan Blanc C\u00f4tes de Gascogne 2020 - Plaimont        dry"
}
[chain/start] [1:chain:RetrievalQA > 2:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 2:chain:StuffDocumentsChain > 3:chain:LLMChain] Entering Chain run with input:
{
  "question": "Is Fleur de d'Artagnan Blanc C\u00f4tes de Gascogne 2020 - Plaimont        dry",
  "context": ": 40\ntitle: Fleur de d'Artagnan Blanc C\u00c3\u00b4tes de Gascogne 2020 - Plaimont\ntype: white wine\nkind: dry\ncountry: France\nregion: Gascogne (FR)\nprice_eur: 4.88\nvolume_lit: 0.75\norder_number: 70011-FR-WW-BL-005-12<<<<>>>>>: 56\ntitle: Fleur de d'Artagnan Blanc C\u00c3\u00b4tes de Gascogne 2020 - Plaimont\ntype: white wine\nkind: dry\ncountry: France\nregion: Gascogne (FR)\nprice_eur: 4.88\nvolume_lit: 0.75\norder_number: 70011-FR-WW-BL-005-12<<<<>>>>>: 24\ntitle: Fleur de d'Artagnan Blan

"Yes, Fleur de d'Artagnan Blanc Côtes de Gascogne 2020 - Plaimont is a dry white wine."

In [21]:
langchain.debug = False

In [22]:
predictions = qa.apply(examples)



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [23]:
from langchain.evaluation.qa import QAEvalChain

In [24]:
llm = ChatOpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [25]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [26]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:
Question: Is Fleur de d'Artagnan Blanc Côtes de Gascogne 2020 - Plaimont        dry
Real Answer: Yes
Predicted Answer: Yes, Fleur de d'Artagnan Blanc Côtes de Gascogne 2020 - Plaimont is a dry white wine.
Predicted Grade: CORRECT

Example 1:
Question: What kind is Fragolino Rosso Frizzante - Bottega?
Real Answer: fragolino
Predicted Answer: Fragolino Rosso Frizzante - Bottega is a fragolino, which is a wine-based beverage.
Predicted Grade: CORRECT

Example 2:
Question: What is the name of the wine-based beverage mentioned in the document and what is its origin?
Real Answer: The name of the wine-based beverage mentioned in the document is Fragolino Rosso Frizzante and it originates from Italy.
Predicted Answer: The name of the wine-based beverage mentioned in the document is Fragolino Rosso Frizzante - Bottega and its origin is Italy.
Predicted Grade: CORRECT

Example 3:
Question: What is the title of the white wine mentioned in the document?
Real Answer: The title of the whi

In [ ]:
#dataset tam olarak bu alanda calismaya uygun degil fakat daha iyisi yapilabilirdi...